- 3-layer LSTM encoder + 3-layer LSTM decoder
- 50 epoches
- Dropout + Recurrent dropout


In [ ]:
!pip install nltk==3.5

     |████████████████████████████████| 1.4MB 29.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=5d3661f859e800cefaf04f86d3e536fd4123a5a8fafb2a761c53d76de74e980c
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
# Test Summarization with LSTM
# Author: Xuan Lin

# Save output to a file
import sys
import datetime as dt

master_ts = dt.datetime.now() - dt.timedelta(hours = 5)
ts_filename = format(master_ts, "%m%d%H%M")

#full_path = '/content/drive/My Drive/Deep Learning Training/'
#orig_stdout = sys.stdout
#f = open(full_path + 'output_'+ts_filename+'.txt', 'w')
#sys.stdout = f

print("Deep Learning Group Project - Text Summarization")
print("Author: Xuan Lin")
print("")
print("File timestamp: {}".format(ts_filename))
print("")

# ------------------- Hyper-Parameters --------------------

weights_to_load = ''
print("Load model weights from {}".format(weights_to_load))

params = dict({'title': '3L50eDORecDO',
               'n_epoch':50,
              'batch_size': 500,
              'optimizer': 'adam',
              'loss_fun': 'sparse_categorical_crossentropy', # categorical_crossentropy
              'max_len_text': 80,
              'max_len_summary': 8,
              'patience': 3,
              'dropout': 0.1,
              'rec_dropout': 0.1
              })
print("Hyper-parameters: {}".format(params))
print("")

# ---------------------- Load Packages -------------------------

#import pkg_resources
#pkg_resources.require("nltk==3.5")
import pandas as pd
import numpy as np
import re
from gensim import models
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import warnings
warnings.filterwarnings("ignore")
import nltk
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn as sns
#nltk.download('stopwords')
from Attention import AttentionLayer

import tensorflow as tf
tf.random.set_seed(2020)

print("Packages Loaded.")
print("")


# --------------------- Load data ------------------------

#path = '/content/drive/My Drive/'
#df = pd.read_csv(path + "Copy of Reviews Cleaned.csv")
full_path = '/content/drive/My Drive/Deep Learning Training/'

#print("Dataset Size: {}".format(df.shape))
#print("")

max_len_text = params['max_len_text']
max_len_summary = params['max_len_summary']

# Load train, val, and test data from csv files
train = pd.read_csv('/content/drive/My Drive/Deep Learning Training/Data Sets/TrainSet.csv')
val = pd.read_csv('/content/drive/My Drive/Deep Learning Training/Data Sets/ValSet.csv')
test= pd.read_csv('/content/drive/My Drive/Deep Learning Training/Data Sets/TestSet80k.csv')

train_id, Xtrain, ytrain = train['id'], train['text'].astype('str'), train['summary'].astype('str')
val_id, Xval, yval = val['id'], val['text'].astype('str'), val['summary'].astype('str')
test_id, Xtest, ytest = test['id'], test['text'].astype('str'), test['summary'].astype('str')
y_true = ytest

"""
# Train Test Split
Xtrain, Xtest, ytrain, ytest = train_test_split(df[['Id','cleaned_text']].astype('str'),
                                              df['cleaned_summary'].astype('str'),
                                              test_size=0.2,random_state = 2020)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size = 0.1, random_state = 2020)
train_id, Xtrain = Xtrain['Id'], Xtrain['cleaned_text']
val_id, Xval = Xval['Id'], Xval['cleaned_text']
test_id, Xtest = Xtest['Id'], Xtest['cleaned_text']
y_true = ytest
"""

print("Train X & y: ", Xtrain.shape, ytrain.shape)
print("Validation X & y: ", Xval.shape, yval.shape)
print("Test X & y: ", Xtest.shape, ytest.shape)
print("")


# ---------------------- Tokenization ----------------------

# Review
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(Xtrain))
Xtrain = x_tokenizer.texts_to_sequences(Xtrain) 
Xval = x_tokenizer.texts_to_sequences(Xval)
Xtest = x_tokenizer.texts_to_sequences(Xtest)

Xtrain = pad_sequences(Xtrain, maxlen=max_len_text, padding='post')
Xval = pad_sequences(Xval, maxlen=max_len_text, padding='post')
Xtest = pad_sequences(Xtest, maxlen=max_len_text, padding='post')
x_voc_size = len(x_tokenizer.word_index) + 1

print("Review tokenization and padding done.")


# Summary
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(ytrain))

ytrain = y_tokenizer.texts_to_sequences(ytrain)
yval = y_tokenizer.texts_to_sequences(yval)
ytest = y_tokenizer.texts_to_sequences(ytest) 

ytrain = pad_sequences(ytrain, maxlen=max_len_summary, padding='post')
yval = pad_sequences(yval, maxlen=max_len_summary, padding='post')
ytest = pad_sequences(ytest, maxlen=max_len_summary, padding='post')
y_voc_size = len(y_tokenizer.word_index) + 1

print("Summary tokenization and padding done.")
print("")

#print("Sample Review: {}".format(Xtrain[0]))



# ------------------ LSTM Training Model -------------------


from keras import backend as K
K.clear_session()

latent_dim = 500   # dimension of embedding

# Encoder ----------

# Inputs are review entries, which has 80 words max
encoder_input = Input(shape=(max_len_text,))
# Create an embedding layer to be trained, input_dim = number of volcabularies/tokens
# output_dim is the dim of the encoded vector
enc_emb = Embedding(input_dim = x_voc_size, output_dim = latent_dim, trainable = True)(encoder_input)

# LSTM Layer 1
# return_sequence: returns hidden state and cell state of every timestamp
# return_state: produce hidden state and cell state of the last timestamp only
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True,
                     activation='tanh', 
                     dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)
#encoder_output1 = BatchNormalization(axis = 1)(encoder_output1)

# LSTM Layer 2
encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True, 
                     dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# LSTM Layer 3
encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True, 
                     dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)


# Decoder ----------

decoder_input = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, latent_dim, trainable = True)
dec_emb = dec_emb_layer(decoder_input)

# LSTM Layer 1
decoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
decoder_output1, decoder_fwd_state1, decoder_back_state1 = decoder_lstm1(dec_emb,
                                                                      initial_state=[state_h, state_c])

# LSTM Layer 2
decoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
decoder_output2, decoder_fwd_state2, decoder_back_state2 = decoder_lstm2(decoder_output1)

# LSTM Layer 3
decoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout = params['dropout'], recurrent_dropout = params['rec_dropout'])
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm3(decoder_output2)


# Attention Layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention output and decoder LSTM output horizontally
decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_outputs, attn_out])

# Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation = 'softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)


# Model -----------

# Define the model
model = Model([encoder_input, decoder_input], decoder_outputs)
print("")
print("------------------ Training Model -------------------")
print("")
model.summary()
if weights_to_load != '':
  model.load_weights(full_path+weights_to_load)


# Training -----------

start_ts = dt.datetime.now()-dt.timedelta(hours=5)
print("Training starts at {}".format(start_ts))

model.compile(optimizer=params['optimizer'], loss=params['loss_fun'])
es = EarlyStopping(monitor='val_loss', mode = 'min', verbose=1, patience = params['patience'], restore_best_weights=True)
history=model.fit([Xtrain, ytrain[:,:-1]], 
                  ytrain.reshape(ytrain.shape[0], ytrain.shape[1], 1)[:,1:],
                  epochs=params['n_epoch'], callbacks=[es], batch_size = params['batch_size'], 
                  validation_data=([Xval, yval[:,:-1]], yval.reshape(yval.shape[0],yval.shape[1], 1)[:,1:]))

end_ts = dt.datetime.now()-dt.timedelta(hours=5)
print("Training ends at {}".format(end_ts))
print("Training Time: {}".format(end_ts - start_ts))
print("")


# Save results -----------

# Save weights
model.save_weights(full_path+'LSTM_weights_'+ts_filename+'.h5')
print("Weights saved.")
print("")

# Save CV results
pd.DataFrame(history.history).to_csv(full_path+'LSTM_history_'+str(ts_filename)+'.csv', index = False)

# Save image
loss_plot = plt.figure()
train_loss, val_loss = history.history['loss'], history.history['val_loss']
plt.plot(history.epoch, train_loss, color = "black")
plt.plot(history.epoch, val_loss, color = "blue")
plt.title("Training and Validation Loss")
plt.legend(['Train', 'Validation'])
#plt.show()
loss_plot.savefig(full_path+'LSTM_loss_'+str(ts_filename)+'.jpg')
plt.close(loss_plot)

print("CV results saved.")
print("")
print("")


# --------------------- LSTM Inference Model -------------------

print("------------------ Inference Model -------------------")

reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y_tokenizer.word_index

# Encoder Inference
encoder_model = Model(inputs=encoder_input,
                      outputs = [encoder_outputs, state_h, state_c])

# Decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text, latent_dim)) # output of encoder

# Get the embeddings of the decoder sequence
dec_emb2 = dec_emb_layer(decoder_input)

# Predict the next word
# set the initial state to the previous state
decoder_outputs_inf1, state_h_inf1, state_c_inf1 = decoder_lstm1(dec_emb2,
                                                   initial_state = [decoder_state_input_h,
                                                                    decoder_state_input_c])
decoder_outputs_inf2, state_h_inf2, state_c_inf2 = decoder_lstm2(decoder_outputs_inf1)
decoder_outputs2, state_h2, state_c2 = decoder_lstm3(decoder_outputs_inf2)

# Attention Inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])

# Concatenate attention output and decoder output
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# Softmax layer to output probability distribution
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(inputs = [decoder_input]+[decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
                      outputs = [decoder_outputs2] + [state_h2, state_c2])

print("")
decoder_model.summary()
print("")


# ------------------- Inference Helper Function -----------------

# Functions to output the summary
# by predicting the next word given the previous one
def decode_sequence(input_seq, debug = False, threshold = 0.8):
  # Encode the input as state vectors
  e_out, e_h, e_c = encoder_model.predict(input_seq)
  # Placeholder of the target sequence
  target_seq = np.zeros((1,1))
  # Choose "_START_" as the first word
  target_seq[0,0] = target_word_index['start']
  stop_condition = False
  decoded_sentence = ''
  # Decode summary
  while stop_condition == False:
    # Get probabilities, hidden state, and memory cell for the next word
    output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
    # Sample the best token
    sampled_token_index = np.array(output_tokens[0,-1,:].argsort()[::-1][:2])

    if sampled_token_index[0] == target_seq:
      sampled_token_index = sampled_token_index[1]
    else:
      sampled_token_index = sampled_token_index[0]

    sampled_token_prob = np.max(output_tokens[0,-1,:])
    sampled_token = reverse_target_word_index[sampled_token_index]

    if debug == True:
      top_3 = np.array(output_tokens[0,-1,:].argsort()[::-1][:3])
      top_3_word = [reverse_target_word_index[n] for n in top_3]
      top_3_prob = sorted(output_tokens[0,-1,:])[-3:][::-1]
      print(top_3_prob)
      #print(output_tokens[0,-1,:])
      print(top_3)
      print(top_3_word)
    # Stop?
    if (sampled_token == 'end' or len(decoded_sentence.split())>=(max_len_summary-1)):
      stop_condition = True
    elif (sampled_token_prob < threshold) & (decoded_sentence != ''):
      stop_condition = True
    else:
      decoded_sentence = decoded_sentence + sampled_token + ' '
    # Update the target sequence
    target_seq[0,0] = sampled_token_index
    # Update inital states
    e_h, e_c = h, c
  return decoded_sentence[:-1]


# Functions to convert index sequence to text
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['end']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    return newString


# ---------------------- Prediction -----------------------

print("")
print("------------------ Prediction -------------------")
print("")

pred_start_ts = dt.datetime.now()-dt.timedelta(hours=5)
print("Prediction starts at {}".format(pred_start_ts))

pred = []
for i in range(len(ytest)):
  pred += [decode_sequence(Xtest[i].reshape(1, max_len_text), threshold = 0.07)]
  if i % 1000 == 0:
    print("Round {} Completed, {}".format(i, dt.datetime.now()))

pred_end_ts = dt.datetime.now()-dt.timedelta(hours=5)
print("Prediction ends at {}".format(pred_end_ts))
print("Prediction takes {}.".format(pred_end_ts - pred_start_ts))
print("")

pred_output = pd.DataFrame({'id': test_id,'y_pred': pred, 
                            'y_true': [' '.join(x.split()[1:-1]) for x in y_true]})

pred_output.to_csv(full_path+'pred_'+str(ts_filename)+'.csv', index = False)


# --------------------- Evaluation -------------------------

import gensim.downloader as api
word2vec = api.load("word2vec-google-news-300")
nltk.download('wordnet')

eval_start = dt.datetime.now()-dt.timedelta(hours=5)
print("Evaluation starts at {}".format(eval_start))


def sentence_bleu_n(x, weights):
  return sentence_bleu(references = [str(x[1]).split()], 
                       hypothesis = str(x[0]).split(),
                       weights = weights)
  
# bleu1
pred_output['bleu1'] = pred_output[['y_pred', 'y_true']].apply(lambda x: sentence_bleu_n(x, weights = [1,0,0,0]), axis=1)

# bleu2
pred_output['bleu2'] = pred_output[['y_pred', 'y_true']].apply(lambda x: sentence_bleu_n(x, weights = [.5,.5,0,0]), axis=1)

# bleu3
pred_output['bleu3'] = pred_output[['y_pred', 'y_true']].apply(lambda x: sentence_bleu_n(x, weights = [.33,.33,.33,0]), axis=1)

# bleu4
pred_output['bleu4'] = pred_output[['y_pred', 'y_true']].apply(lambda x: sentence_bleu_n(x, weights = [.25,.25,.25,.25]), axis=1)

# WMDistance
get_wmd = lambda x: word2vec.wmdistance(str(x[1]), str(x[0]))
pred_output['wmd'] = pred_output[['y_pred', 'y_true']].apply(get_wmd, axis=1)

pred_output.to_csv(full_path+'pred_'+str(ts_filename)+'.csv', index = False)

eval_end = dt.datetime.now()-dt.timedelta(hours=5)
print("Evaluation end at {}".format(eval_end))
print("Evaluation takes {}".format(eval_end - eval_start))
print("")

# Metrics Overview
eval_overview = pred_output[['bleu1', 'bleu2', 'bleu3', 'bleu4', 'meteor', 'wmd']].describe().iloc[1:]
print("Evaluation Metrics Overview")
print("")
print(eval_overview)
print("")

eval_overview.to_csv(full_path + 'eval_' + str(ts_filename) + '.csv', index = False)

# Visualize Metrics
plot_eval = pd.DataFrame(pred_output[['bleu1', 'bleu2', 'bleu3', 'bleu4', 'meteor', 'wmd']].unstack()).reset_index().drop(columns = 'level_1')
plot_eval.rename(columns={'level_0':'Metrics', 0: 'Score'}, inplace = True)

eval_plot = plt.figure()
sns.boxplot(x='Metrics', y ='Score', data = plot_eval)
plt.title('Evaluation Metrics')
eval_plot.savefig(full_path + 'evaluation_'+ts_filename+'.jpg')
plt.close(eval_plot)



Deep Learning Group Project - Text Summarization
Author: Xuan Lin

File timestamp: 06030018

Load model weights from 
Hyper-parameters: {'title': '3L50eDORecDO', 'n_epoch': 50, 'batch_size': 500, 'optimizer': 'adam', 'loss_fun': 'sparse_categorical_crossentropy', 'max_len_text': 80, 'max_len_summary': 8, 'patience': 3, 'dropout': 0.1, 'rec_dropout': 0.1}



Using TensorFlow backend.


Packages Loaded.

Train X & y:  (284120,) (284120,)
Validation X & y:  (31569,) (31569,)
Test X & y:  (16000,) (16000,)

Review tokenization and padding done.
Summary tokenization and padding done.


------------------ Training Model -------------------

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 500)      51082500    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 80, 500), (N 2002000     embedding[0][0]                  
_____________________________________